# With CV

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
data_ndarr = np.load(r"../../Data/all_pids_q=0.3.npy", allow_pickle=True)
datadict = data_ndarr.item()

In [3]:
p_ids = datadict.keys()
smaple_size_dict={}
print("patients: n =", len(p_ids), end='\n\n')

for p_id in p_ids:
    df = datadict[p_id]
    # drop column from input data file
    #df.drop(['id','date','am_pef_org','BMI','sex'], axis=1, inplace=True) #ALL DROPS SHOULD OCCUR HERE
    df.drop(['id','date','am_pef_org','BMI','sex', 'age'], axis=1, inplace=True) #,'id','date','am_pef_org','BMI',  'sex', 'age'
    df = np.asarray(df).astype(np.float32)
    print(p_id, "shape:", df.shape)#, end="\t\t")
    smaple_size_dict[p_id]=df.shape[0]

datadict[p_id].head(4)

patients: n = 26

SB-001 shape: (188, 28)
SB-003 shape: (180, 28)
SB-008 shape: (178, 28)
SB-011 shape: (210, 28)
SB-012 shape: (194, 28)
SB-022 shape: (204, 28)
SB-023 shape: (194, 28)
SB-025 shape: (117, 28)
SB-028 shape: (92, 28)
SB-033 shape: (53, 28)
SB-037 shape: (210, 28)
SB-043 shape: (191, 28)
SB-056 shape: (191, 28)
SB-059 shape: (203, 28)
SB-060 shape: (88, 28)
SB-070 shape: (202, 28)
SB-071 shape: (124, 28)
SB-073 shape: (161, 28)
SB-078 shape: (153, 28)
SB-079 shape: (142, 28)
SB-080 shape: (181, 28)
SB-081 shape: (34, 28)
SB-082 shape: (137, 28)
SB-083 shape: (206, 28)
SB-089 shape: (49, 28)
SB-112 shape: (180, 28)


,income,road_dist,cooking,y_am_pef,tempin,humidin,pm25in,co2in,tempdiffin,humidiffin,...,windsd,humid,varp,dewpt,airp,seap,solrhr,solramnt,grdt,class
1,0.857,0.0,0.167,0.558,0.216,0.513,0.358,0.678,0.562,0.364,...,0.107,0.668,0.174,0.471,0.822,0.819,0.165,0.003,0.206,0.0
2,0.857,0.0,0.167,0.006,0.299,0.521,0.411,0.700,0.436,0.672,...,0.171,0.388,0.190,0.491,0.676,0.670,0.603,0.165,0.383,0.0
3,0.857,0.0,0.167,0.078,0.420,0.713,0.372,0.918,0.469,0.821,...,0.515,0.359,0.197,0.481,0.519,0.518,0.485,0.104,0.346,1.0
4,0.857,0.0,0.167,0.428,0.438,0.468,0.224,0.762,0.560,0.490,...,0.338,0.527,0.175,0.467,0.596,0.596,0.491,0.141,0.339,0.0


In [4]:
print(datadict[p_id].columns)
print(datadict[p_id].shape)

Index(['income', 'road_dist', 'cooking', 'y_am_pef', 'tempin', 'humidin',
       'pm25in', 'co2in', 'tempdiffin', 'humidiffin', 'pm25diffin', 'pm10',
       'pm25', 'o3', 'no2', 'co', 'so2', 'temp', 'windsd', 'humid', 'varp',
       'dewpt', 'airp', 'seap', 'solrhr', 'solramnt', 'grdt', 'class'],
      dtype='object')
(180, 28)


In [5]:
target_grp = ["SB-001", "SB-003", "SB-008", "SB-011", "SB-012", "SB-022", "SB-023", "SB-025", "SB-028", "SB-037", "SB-043", "SB-056", "SB-059", "SB-060", "SB-070", "SB-071", "SB-073", "SB-078", "SB-079", "SB-080", "SB-082", "SB-083", "SB-112"]

print(len(target_grp))


23


In [6]:
from TL_Packages import *

# Architecture_GROUP = "TL"
Architecture_GROUP = "model"
RESULT_DIR = 'Results/'
EVALUATION_METRICS = ["Weighted Accuracy", "Sensitivity/Recall", "Specificity", "Precision_class0", "Precision_class1", "Precision_avg", "F1_class0", "F1_class1", "F1_avg", "auc_roc_score", "False_Discovery_Rate", "False_Negative_Rate", "False_Omission_Rate", "False_Positive_Rate", "Jaccard"]

2023-08-24 23:41:17.114400: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 23:41:17.167834: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 23:41:17.168760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 23:41:17.971984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
def TL_Base_Model(POPULATION_EPOCH_SIZE,PATIENT_EPOCH_SIZE, p_ids):
    minority_variable=0
  #Target MOdel dataframe
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)
  #Population model results
    pop_results=pd.DataFrame(columns=EVALUATION_METRICS)
    minor_var=minority_variable
    for target in tqdm(p_ids):
        if target not in target_grp:
            continue
        runtime()
        print("~~~~~~~~~~~", POPULATION_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", PATIENT_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", target, "~~~~~~~~~~~~~~~~~~~")

        patient_dir = RESULT_DIR + "/diagrams/target" 

        # create directory for this patient
        if not (patient_dir and patient_dir):
            os.makedirs(patient_dir)
        CLASS_VAR='class'
        sample_size=smaple_size_dict[target]
        # load population data
        X_train, X_test, Y_train, Y_test = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var,test_frac=0.05,
                                                      transfer=True, balance=True)
        print(f"X_train = {len(X_train)}, y_train = {len(Y_train)}")
        assert len(X_train) == len(Y_train), "X_train and Y_train have different lengths."
        #num_folds = min(5, len(X_train))
        #print(num_folds)
        # k-fold cross-validation for population model
        kf_pop = KFold(n_splits=3, shuffle=True)
        population_models = []
        fold_number=0
        for train_idx, val_idx in kf_pop.split(X_train):
             # Check if the indices are within bounds
            assert train_idx.max() < len(X_train), f"train_idx exceeds bounds of X_train (size: {len(X_train)})"
            assert val_idx.max() < len(X_train), f"val_idx exceeds bounds of X_train (size: {len(X_train)})"


            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            Y_train_fold, Y_val_fold = Y_train[train_idx], Y_train[val_idx]
            print(f"Population Fold {fold_number}: Training data size = {len(X_train_fold)}, Validation data size = {len(X_val_fold)}")
            net = network_tl(X_train_fold, Y_train_fold, n_hidden=32, learning_rate=0.001, device='cpu')
            net.set_Y_tind(Y_val_fold)
            ##creating population model
            net.make_pop()
            net.train(n_iters=POPULATION_EPOCH_SIZE)
            #val_predictions = net.nn_pop.predict(X_val_fold)
            #val_loss = net.calculate_loss(val_predictions)

            #population_models.append((net, val_loss))


        # Select the best population model based on validation performance (lowest mean validation loss)
        #best_pop_model, _ = min(population_models, key=lambda x: np.mean(x[1]))
        # Save the best population model
        ## save the fold's population model
            VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD_{fold_number}'
            #best_pop_model_name = f"{target}-best_{VERSION_TAG}.hdf5"
            #best_pop_model_path = patient_dir / best_pop_model_name
            #best_pop_model.save_model(best_pop_model_path)
            #save model
            modelname = f"{target}-{VERSION_TAG}.hdf5"
            model_path= patient_dir + "/" + modelname
            net.nn_pop.save(model_path) #nn_pop
          #save training curve
            training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_training_curve_{target}'
            net.save_training_curve(training_curve_file)
          ## calculate results for population model
            results, c_matrix = net.pop_test(X_val_fold, Y_val_fold)
           # load target patient data

            dataset_list = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var, #explanatory_var
                                balance=True)
           # setup for k-fold cross-validation
            kfold_evaluation_results_list = []
            kfold_confusion_matrix_list = []

            for i in range(2):
            # load the target data in the current round
              X_train_ind, X_test_ind, Y_train_ind, Y_test_ind = dataset_list[i]

            # Make and train a copy of the population model with
            # all but the last layer frozen.
              net.make_ind()
              net.retrain(X_train_ind, Y_train_ind, n_iters=PATIENT_EPOCH_SIZE)
              print("done RE-training", i, "of 3", end=". ")

            # test
              VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD{fold_number}_IND_FOLD{i}'
            #save model
              modelname = f"{target}-{VERSION_TAG}.hdf5"
              model_path= patient_dir + modelname
              net.nn_ind.save(model_path) #nn_pop

              #save retraining curve
              training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_retraining_curve_{target}'
              net.save_training_curve(training_curve_file)

            # print("Calling net.test Now")
            # print("X_test",X_test)
            # print("Y_test",Y_test)
            ## results for individual model for this particular fold
              results, c_matrix = net.test(X_test_ind, Y_test_ind)
            # print("results",results)
            ## adding up the results(evaluation metrics) of all the folds of individual model
              kfold_evaluation_results_list += [results]
            ## adding up the results of confusion matrix for all the folds of individual model
              kfold_confusion_matrix_list += [c_matrix]

              print("done testing", i, "of 2.")

            ## version tag for target model confusion matrix
            TARGET_VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_target_POP_FOLD_{fold_number}'
        ##save file tag for target model of particular pop fold
            target_confusion_matrixes_file = (f"{str(patient_dir)}/{TARGET_VERSION_TAG}_confusion_matrix_{target}")
        ###save file tag for Population model of particular pop fold
            #population_confusion_matrixes_file = (f"{str(patient_dir)}/{VERSION_TAG}_confusion_matrix_{target}")
        ## dataframe for the average of all the indivudial models results
            avg_results = pd.DataFrame(kfold_evaluation_results_list, columns=EVALUATION_METRICS).mean().values
            avg_results = avg_results.reshape(1, 15)
            ##dataframe for population model results for a particular pop model fold'
            results=results.reshape(1,15)
            pop_results=pd.DataFrame(results, columns=EVALUATION_METRICS)
            pop_results.index = [f"{target}_{sample_size}_size_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD_{fold_number}"]
            ## dataframe for individual model average results for a particular population fold
            df = pd.DataFrame(avg_results, columns=EVALUATION_METRICS)
            df.index = [f"{target}_{sample_size}_size_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_Target_With_POP_FOLD_{fold_number}"]
            ## concatinating the pop results dataframe and avg target results dataframe for a particular population fold
            df_evaluation_results = pd.concat([df_evaluation_results,pop_results,df], axis = 0)
            ##saving target model confusion matrix
            net.save_confusion_matrix(kfold_confusion_matrix_list, target_confusion_matrixes_file)
            ##saving population model confusion matrix
            #net.save_confusion_matrix(c_matrix, population_confusion_matrixes_file)
            fold_number+=1

    # output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics_{sorted(set(p_ids).difference(target_grp))[0]}.csv')
    output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics.csv')

    if os.path.isfile(output_path):
        df_evaluation_results.to_csv(output_path,
                                        mode='a',
                                        header=False
                                    )
    else:
                df_evaluation_results.to_csv(output_path,
                                        mode='a'
                                    )

    print("evaluation results saved for", VERSION_TAG)

In [8]:
POPULATION_EPOCH_SIZE = [*range(500, 1600, 500)]
PATIENT_EPOCH_SIZE = [*range(500, 1600, 500)]   

print("POPULATION_EPOCH_SIZE:",POPULATION_EPOCH_SIZE, "Len:", len(POPULATION_EPOCH_SIZE))
print("PATIENT_EPOCH_SIZE:",PATIENT_EPOCH_SIZE, "Len:", len(PATIENT_EPOCH_SIZE))
print("target_grp:",target_grp, "Len:", len(target_grp))
print("Total records in each file:",len(POPULATION_EPOCH_SIZE) * len(PATIENT_EPOCH_SIZE) * len(target_grp))

POPULATION_EPOCH_SIZE: [500, 1000, 1500] Len: 3
PATIENT_EPOCH_SIZE: [500, 1000, 1500] Len: 3
target_grp: ['SB-001', 'SB-003', 'SB-008', 'SB-011', 'SB-012', 'SB-022', 'SB-023', 'SB-025', 'SB-028', 'SB-037', 'SB-043', 'SB-056', 'SB-059', 'SB-060', 'SB-070', 'SB-071', 'SB-073', 'SB-078', 'SB-079', 'SB-080', 'SB-082', 'SB-083', 'SB-112'] Len: 23
Total records in each file: 207


In [9]:
from datetime import date, datetime
def runtime():
    print("Runtime:", datetime.now().hour, "o' clock and", datetime.now().minute, "minutes")

def run_tlsi(file):
    """
        run transfer learning algorithm on sample imbalanced data set
    """
    pass

In [10]:
TL_Base_Model(100,300, p_ids)# test this one after new value

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 23 o' clock and 41 minutes
~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
Before balancing count  1.0    2659
0.0    1021
Name: class, dtype: int64


2023-08-24 23:41:19.846806: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


~~~~~~~ value_counts: ~~~~~~~
SHOULD BE BALANCED NOW: 1.0    2659
0.0    2659
Name: class, dtype: int64
X_train = 5318, y_train = 5318
Population Fold 0: Training data size = 3545, Validation data size = 1773


/home/st/sbhuvaji/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

56/56 [==============================] - 0s 815us/step
cm [[643 220]
 [235 666]]
cm.ravel() [643 220 235 666]
Before balancing count  1.0    76
0.0    14
Name: class, dtype: int64
~~~~~~~ value_counts: ~~~~~~~
SHOULD BE BALANCED NOW: 0.0    76
1.0    76
Name: class, dtype: int64
Before balancing count  1.0    65
0.0    25
Name: class, dtype: int64
~~~~~~~ value_counts: ~~~~~~~
SHOULD BE BALANCED NOW: 0.0    65
1.0    65
Name: class, dtype: int64
done RE-training 0 of 3. 

/home/st/sbhuvaji/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 0s 1ms/step
cm [[ 9 16]
 [13 52]]
cm.ravel() [ 9 16 13 52]
done testing 0 of 2.
3/3 [==============================] - 0s 1ms/step
cm [[ 9  5]
 [25 51]]
cm.ravel() [ 9  5 25 51]
done testing 1 of 2.


/home/st/sbhuvaji/1AllP/Individual Patient_All Models/TLandTL/Actual TL/HeavyDecoder/WithCV/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/home/st/sbhuvaji/1AllP/Individual Patient_All Models/TLandTL/Actual TL/HeavyDecoder/WithCV/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')
100%|██████████| 26/26 [1:10:23<00:00, 162.43s/it]

evaluation results saved for TLSI_model_100_300_POP_FOLD2_IND_FOLD1


In [11]:
print("Done")

Done
